<a href="https://colab.research.google.com/github/Joyce-ZhouY/ECE1512-ProjectA/blob/main/Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
from PIL import Image
import os
import numpy as np
import pandas as pd
from typing import Union
import time
import matplotlib.pyplot as plt

# **Load and process data**

In [ ]:
! unzip /content/images.zip

In [ ]:
#mkdir
path_train = '/content/train'
tf.io.gfile.mkdir(
    path_train
)

path_test = '/content/test'
tf.io.gfile.mkdir(
    path_test
)

path_train_HP = '/content/train/HP'
tf.io.gfile.mkdir(
    path_train_HP
)

path_train_SSA = '/content/train/SSA'
tf.io.gfile.mkdir(
    path_train_SSA
)

path_test_HP = '/content/test/HP'
tf.io.gfile.mkdir(
    path_test_HP
)

path_test_SSA = '/content/test/SSA'
tf.io.gfile.mkdir(
    path_test_SSA
)

path_train_aug = '/content/train_aug'
tf.io.gfile.mkdir(
    path_train_aug
)

path_train_HP_aug = '/content/train_aug/HP'
tf.io.gfile.mkdir(
    path_train_HP_aug
)

path_train_SSA_aug = '/content/train_aug/SSA'
tf.io.gfile.mkdir(
    path_train_SSA_aug
)

# **Data Augmentation**

In [ ]:
path = '/content/annotations.csv'
df = pd.read_csv(path)
csv = dict(df)
image_num = len(csv.get('Image Name'))

# print(csv.get('Image Name')[0])

for i in range(image_num):
  if csv.get('Partition')[i] == "train":
    if csv.get('Majority Vote Label')[i] == "HP":
      src = '/content/images/' + csv.get('Image Name')[i]
      dst = '/content/train/HP/' + csv.get('Image Name')[i]
      tf.io.gfile.copy(src, dst, overwrite=True)
    else:
      src = '/content/images/' + csv.get('Image Name')[i]
      dst = '/content/train/SSA/' + csv.get('Image Name')[i]
      tf.io.gfile.copy(src, dst, overwrite=True)
  else:
    if csv.get('Majority Vote Label')[i] == "HP":
      src = '/content/images/' + csv.get('Image Name')[i]
      dst = '/content/test/HP/' + csv.get('Image Name')[i]
      tf.io.gfile.copy(src, dst, overwrite=True)
    else:
      src = '/content/images/' + csv.get('Image Name')[i]
      dst = '/content/test/SSA/' + csv.get('Image Name')[i]
      tf.io.gfile.copy(src, dst, overwrite=True)

In [ ]:
import matplotlib. image as image
from matplotlib import pyplot as plt
from random import random

path = '/content/annotations.csv'
df = pd.read_csv(path)
csv = dict(df)
image_num = len(csv.get('Image Name'))
#print("num=",image_num)

for i in range(image_num):
    if csv.get('Majority Vote Label')[i] == "HP":
      src = '/content/images/' + csv.get('Image Name')[i]
      img=image.imread(src)
      img_aug = tf.image.rot90(tf.image.random_flip_up_down(img),k=round(random()*3))+tf.random.normal(shape=[224,224,3], mean=0.0, stddev=0.1,dtype=tf.float32)
      tf.keras.utils.save_img('/content/train_aug/HP/'+ csv.get('Image Name')[i], img_aug, data_format=None, file_format=None, scale=True)
    else:
      src = '/content/images/' + csv.get('Image Name')[i]
      img=image.imread(src)
      img_aug = tf.image.rot90(tf.image.random_flip_up_down(img),k=round(random()*3))+tf.random.normal(shape=[224,224,3], mean=0.0, stddev=0.1,dtype=tf.float32)
      tf.keras.utils.save_img('/content/train_aug/SSA/'+ csv.get('Image Name')[i], img_aug, data_format=None, file_format=None, scale=True)
      #break

In [ ]:
IMG_SIZE = (224,224)
# train_dir = '/content/train'
train_dir = '/content/train_aug'
BATCH_SIZE=32
train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir,
                                                            shuffle=True,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE)

test_dir = '/content/test' 
test_dataset = tf.keras.utils.image_dataset_from_directory(test_dir,
                                                            shuffle=True,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE)

In [ ]:
train_dataset = train_dataset.map(lambda x, y: (tf.cast(x, tf.float32), tf.one_hot(y, 2)))
test_dataset = test_dataset.map(lambda x, y: (tf.cast(x, tf.float32), tf.one_hot(y, 2)))

# **Accuracy matrix**

In [ ]:
from keras import backend as K
# The following code is from answers in this website:
# https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))
  


# Create Teacher Model
# -- use pre-trained ResNet50V2

In [ ]:
# load pre-trained model -- Techer
# Create base model
teacher_resnet_50 = tf.keras.applications.resnet_v2.ResNet50V2(include_top=False, weights='imagenet')

# Freeze base model
teacher_resnet_50.trainable=False

# Create new model on top.
inputs = tf.keras.Input(shape=(224, 224, 3))
x = teacher_resnet_50(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D(dtype='float32')(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(2)(x)
teacher_model = tf.keras.Model(inputs, outputs)

# loss_fn = tf.nn.sigmoid_cross_entropy_with_logits
# loss_fn = tf.nn.sparse_softmax_cross_entropy_with_logits 
# loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True),
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

initial_epoch = 10
teacher_model.compile(optimizer=optimizer, 
                      loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
                      metrics=['acc',f1_m,precision_m, recall_m])
teacher_model.summary()


# Train Teacher Model

In [ ]:
history = teacher_model.fit(train_dataset, epochs=initial_epoch)

In [ ]:
acc = history.history['acc']
f1 = history.history['f1_m']
print(f1)
precision = history.history['precision_m']
recall = history.history['recall_m']
loss = history.history['loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Accuracy',linestyle='--', marker='o')
plt.plot(f1, label='F1 Score',linestyle='--', marker='o')
plt.plot(recall, label='Recall',linestyle='--', marker='o')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
# plt.plot([initial_epoch-1,initial_epoch-1],
#           plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training Accuracy of Teacher Model')

plt.subplot(2, 1, 2)

plt.ylim([10, 60])
plt.plot(loss, label='Loss',linestyle='--', marker='o')
# # plt.plot([initial_epoch-1,initial_epoch-1],
# #          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training Loss of Teacher Model')
plt.xlabel('epoch')
plt.ylabel('Loss')
# plt.savefig('/content/images_mhist/teacher_train_acc.png')
plt.show()


In [ ]:
test_his = teacher_model.evaluate(test_dataset)

# Fine tune teacher model and re-train

In [ ]:
# unfreeze the final set of CONV layers and make them trainable
teacher_resnet_50.trainable = True
# print("Number of layers in the base model: ", len(student_base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 100

for layer in teacher_resnet_50.layers[:fine_tune_at]:
  layer.trainable = False

teacher_model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),  # Very low learning rate
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['acc',f1_m,precision_m, recall_m])

teacher_model.summary()
print(len(teacher_model.trainable_variables))

# train the model again, this time fine-tuning *both* the final set
# of CONV layers along with our set of FC layers

history_finetune = teacher_model.fit(train_dataset, epochs=25)

In [ ]:
acc += history_finetune.history['acc']
f1 += history_finetune.history['f1_m']
precision += history_finetune.history['precision_m']
recall += history_finetune.history['recall_m'] 
loss += history_finetune.history['loss']


In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Accuracy',linestyle='--', marker='o')
plt.plot(f1, label='F1 Score',linestyle='--', marker='o')
plt.plot(recall, label='Recall',linestyle='--', marker='o')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.plot([initial_epoch,initial_epoch],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training Accuracy of Teacher Model')

plt.subplot(2, 1, 2)

plt.ylim([0, 50])
plt.plot(loss, label='Loss',linestyle='--', marker='o')
plt.plot([initial_epoch,initial_epoch],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training Loss of Teacher Model')
plt.xlabel('epoch')
plt.ylabel('Loss')
# plt.savefig('/content/images_mhist/teacher_train_acc2.png')
plt.show()


**Create student model -- use pre-trained MobileNetV2**

In [ ]:
# student model
def create_student():
  input_size = (224,224,3)
  student_base_model = tf.keras.applications.MobileNetV2(input_shape=input_size,
                                                include_top=False,
                                                weights='imagenet')

  image_batch, label_batch = next(iter(train_dataset))
  feature_batch = student_base_model(image_batch)
  print(feature_batch.shape)

  student_base_model.trainable = False
  global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
  feature_batch_average = global_average_layer(feature_batch)
  prediction_layer = tf.keras.layers.Dense(2)
  prediction_batch = prediction_layer(feature_batch_average)
  print(prediction_batch.shape)

  inputs = tf.keras.Input(shape=input_size)
  x = student_base_model(inputs, training=False)
  x = tf.keras.layers.GlobalAveragePooling2D()(x)
  x = tf.keras.layers.Dropout(0.2)(x)
  outputs = prediction_layer(x)
  student_model = tf.keras.Model(inputs, outputs)
  # student_model.summary()
  student_model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['acc',f1_m,precision_m, recall_m])
  return student_model, student_base_model



# **compute Student Loss**

In [ ]:
# calculate loss to train student
ALPHA = 0.5
TEMPERATURE = 4


# loss = ALPHA * student_hard_target + (1-ALPHA) * KLDivergence(teacher_soft_target, student_soft_target)

def distillation_loss(teacher_logits: tf.Tensor, student_logits: tf.Tensor,
                      temperature: Union[float, tf.Tensor]):
  soft_targets = tf.nn.softmax(teacher_logits / temperature)

  return tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
          soft_targets, student_logits / temperature)) * temperature ** 2


def compute_student_loss(images, labels):
  student_subclass_logits = teacher_model(images, training=True)
  teacher_subclass_logits = student_model(images, training=False)
  distillation_loss_value = distillation_loss(teacher_subclass_logits,student_subclass_logits,DISTILLATION_TEMPERATURE)
  cross_entropy_loss_value = tf.keras.losses.categorical_crossentropy(labels, student_subclass_logits,  from_logits=True)
  loss= ALPHA*distillation_loss_value + (1-ALPHA)* cross_entropy_loss_value
  return loss

def compute_num_correct(model, images, labels):
  class_logits = model(images, training=False)
  value= tf.reduce_sum(
      tf.cast(tf.math.equal(tf.argmax(class_logits, -1), tf.argmax(labels, -1)),
              tf.float32)), tf.argmax(class_logits, -1), tf.argmax(labels, -1)
  return value

def train_and_evaluate(model, compute_loss_fn):
  # optimizer = tf.optimizers.Adam(learning_rate=1e-3)
  accuracy_matrix={}
  time_matrix=[]
  f1 = []
  for epoch in range(1, NUM_EPOCHS + 1):
    # Run training.
    start_time = time.time()
    print('Epoch {}: '.format(epoch), end='')
    for images, labels in train_dataset:
      with tf.GradientTape() as tape:
         # your code start from here for step 4
        loss_value = compute_loss_fn(images,labels)

      grads = tape.gradient(loss_value, model.trainable_variables)
      optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # Run evaluation.
    num_correct = 0
    num_total = 977
    for images, labels in test_dataset:
      # print("num_correct=",num_correct)
      num,a, b = compute_num_correct(model, images, labels)
      num_correct += num
    print("Class_accuracy: " + '{:.2f}%'.format(
        num_correct / num_total * 100))
    accuracy_matrix.update({"Student Accuracy": num_correct / num_total * 100})
    epoch_time = time.time() - start_time
    time_matrix.append(epoch_time)
    print("Total time per epoch: {0:.2f} seconds".format(epoch_time))
    y_pred = student_model(images)
    y_true = labels
    f1_score = f1_m(y_true, y_pred).numpy()
    f1.append(f1_score)
    print("F1_Score: " + '{:.2f}'.format(f1_score))
  total_time = np.sum(time_matrix)
  print("Total time of training this model: {0:.2f} seconds".format(total_time))
  return accuracy_matrix, f1


# **Train and evaluate student model**

In [ ]:
NUM_EPOCHS= 10
optimizer = tf.optimizers.Adam(learning_rate=1e-3)
student_model, student_base_model = create_student()
student_model.summary()
# train and save student model
checkpoint_path = '/content/student_model.ckpt'
model_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
# matrix = train_and_evaluate(student_model,compute_student_loss)
matrix = student_model.fit(train_dataset,
                           epochs = NUM_EPOCHS,
                           callbacks=[model_callback])




# Fine tune student model 

In [ ]:
def fine_tune(model, base_model):
  # unfreeze the final set of CONV layers and make them trainable
  base_model.trainable = True
  # print("Number of layers in the base model: ", len(student_base_model.layers))

  # Fine-tune from this layer onwards
  fine_tune_at = 100

  for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

  optimizer = tf.optimizers.Adam(learning_rate=1e-4)
  model.compile(optimizer=optimizer, 
                loss = None,
                # metrics=['acc'])
                metrics = None)

  # model.summary()
  return model

In [ ]:
NUM_EPOCHS= 25
ALPHA = 0.5
DISTILLATION_TEMPERATURE = 4
optimizer = tf.optimizers.Adam(learning_rate=1e-4)
student_model = fine_tune(student_model, student_base_model)
matrix_fine, f1_fine = train_and_evaluate(student_model,compute_student_loss)


# **Test accuracy vs. Tempreture curve**

In [ ]:
Temperatures = [1,2,4,16,32,64]
ALPHA=0.5
student_accuracy_matrixes = []
student_f1_scores = []
# latest = tf.train.latest_checkpoint(checkpoint_path)
for t in Temperatures:
  TEMPERATURE = t
  tf.keras.backend.clear_session()
  student_model,student_base_model = create_student()
  student_model.load_weights(checkpoint_path)
  # fine tuning student model with different T
  NUM_EPOCHS= 25
  optimizer = tf.optimizers.Adam(learning_rate=1e-4)
  student_model = fine_tune(student_model, student_base_model)
  matrix_fine,f1_fine = train_and_evaluate(student_model,compute_student_loss)
  student_accuracy_matrixes.append(matrix_fine.get('Student Accuracy').numpy())
  student_f1_scores.append(f1_fine)
  

In [ ]:
print(student_f1_scores)
student_f1_scores = [max(f1) * 100 for f1 in student_f1_scores]
# student_f1_scores = [f1 * 100 for f1 in student_f1_scores]

print(student_f1_scores)
print(student_accuracy_matrixes)

plt.title('Student test accracy vs temperature')
plt.xlabel('temperature')
plt.ylabel('Student Accuracy')
xi = list(range(len(Temperatures)))
plt.plot(xi,student_accuracy_matrixes,linestyle='--', marker='o', color='b', label='Accuracy')
plt.plot(xi,student_f1_scores ,linestyle='--', marker='o', color='r', label='F1 Scores')
plt.legend(loc='upper right')
plt.xticks(xi, Temperatures)
# plt.savefig('/content/images_mhist/studentAcc_vs_temperatures.png')
plt.show()

# Comparing the teacher and student model 
## (number of of parameters and FLOPs)

In [ ]:
!pip install keras-flops
from keras_flops import get_flops

In [ ]:
# compare teacher and student model

# count number of parameters of Teacher and student model
teacher_model.summary() # Trainable params: 
student_model.summary()  # Trainable params:

# calculate FLOPs of the teacher and student model
teacher_flops = get_flops(teacher_model, batch_size=BATCH_SIZE)
print("The number of FLOPs of the teacher model:", teacher_flops)
# 223623992512

student_flops = get_flops(student_model, batch_size=BATCH_SIZE)
print("The number of FLOPs of the student model:", student_flops)
# 19604230144

# **Train student from scratch**

In [ ]:
from tensorflow.python import train
# student from scratch
input_size = (224,224,3)
epoch = 10
student_base_model = tf.keras.applications.MobileNetV2(input_shape=input_size,
                                               include_top=False,
                                               weights='imagenet')
image_batch, label_batch = next(iter(train_dataset))
feature_batch = student_base_model(image_batch)


student_base_model.trainable = False
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
prediction_layer = tf.keras.layers.Dense(2)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

inputs = tf.keras.Input(shape=input_size)
x = student_base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
student_no_distillation = tf.keras.Model(inputs, outputs)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
student_no_distillation.compile(optimizer=optimizer, 
                      loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
                      metrics=['acc',f1_m,precision_m, recall_m])


student_no_distillation.summary()
student_no_distillation.fit(train_dataset, epochs=epoch)

In [ ]:
# fine tuining the no distillation student model
student_base_model.trainable = True
# print("Number of layers in the base model: ", len(student_base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 100

for layer in student_base_model.layers[:fine_tune_at]:
  layer.trainable = False

optimizer=tf.keras.optimizers.Adam(1e-4)
epoch = 25
student_no_distillation.compile(optimizer=optimizer, 
                      loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
                      metrics=['acc'])

student_no_distillation.summary()
student_no_distillation.fit(train_dataset, epochs=epoch)
# student_base_model.summary()
